<a href="https://colab.research.google.com/github/albisbub/RFIC-Pitches/blob/master/_notebooks/2020-04-29-ALK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RFIC MEETING 7
> Alaska Airlines Buy Pitch

- toc:true
- branch: master
- badges: true
- comments: true
- author: Blaise and Abby
- permalink: /ALK/
- categories: [fastpages, jupyter]

In [596]:
#hide
#Requirements:
import warnings
warnings.filterwarnings('ignore')
!pip install requests bs4 pandas yahoofinancials yahoo_fin requests_html

In [0]:
#hide
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
from datetime import timedelta, datetime, date
%config InlineBackend.figure_format = 'retina'

chart_width = 550
chart_height= 400

import gspread
from oauth2client.service_account import ServiceAccountCredentials

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import requests
# !pip install 
from bs4 import BeautifulSoup
import pandas as pd
from yahoofinancials import YahooFinancials
from yahoo_fin.stock_info import get_quote_table, get_stats, get_live_price, get_income_statement, get_balance_sheet, get_cash_flow
# !pip install dict
# from dict import dict
def getQuote(ticker):
    return str(round(get_live_price(ticker), 2))

def finviz_summaryTable(symbols):
    # Get Column Header
    req = requests.get("https://finviz.com/quote.ashx?t=FB")
    soup = BeautifulSoup(req.content, 'html.parser')
    table = soup.find_all(lambda tag: tag.name=='table')
    rows = table[8].findAll(lambda tag: tag.name=='tr')
    out=[]
    for i in range(len(rows)):
        td=rows[i].find_all('td')
        out=out+[x.text for x in td]

    ls=['Ticker','Sector','Sub-Sector','Country']+out[::2]

    dict_ls={k:ls[k] for k in range(len(ls))}
    df=pd.DataFrame()
    print('Fetching Data...')
    for j in range(len(symbols)):

      try:

        req = requests.get("https://finviz.com/quote.ashx?t="+symbols[j])
        if req.status_code !=200:
            continue
        soup = BeautifulSoup(req.content, 'html.parser')
        table = soup.find_all(lambda tag: tag.name=='table')
        rows=table[6].findAll(lambda tag: tag.name=='tr')
        sector=[]
        for i in range(len(rows)):
            td=rows[i].find_all('td')
            sector=sector+[x.text for x in td]
        sector=sector[2].split('|')
        rows = table[8].findAll(lambda tag: tag.name=='tr')
        out=[]
        for i in range(len(rows)):
            td=rows[i].find_all('td')
            out=out+[x.text for x in td]
        out=[symbols[j]]+sector+out[1::2]
        out_df=pd.DataFrame(out).transpose()
        df=df.append(out_df,ignore_index=True)
        print('Done!')
        
        df=df.rename(columns=dict_ls)   

      except BaseException as E:
        print("Could not find data:", E)

    return(df)
    
def yf_summaryTable(tickerList):
    values = []
    for i in tickerList:
      try:
        value = get_quote_table(i , dict_result = True)
        values.append(value)
      except BaseException as e:
        print(e)
             
    outD = dict(zip(tickerList,values))
            
    return outD
        
def yf_stats_compTable(tickerList):
    values = []
    for i in tickerList:
      try:
        value = get_stats(i)
        values.append(value)
      except BaseException as e:
        print(e)
             
    outD= dict(zip(tickerList,values))
            
    return outD
               
def get_prices(tickerlist, start, end, period):
    yahoo_financials = YahooFinancials(tickerlist)
    jsonDATA = yahoo_financials.get_historical_price_data(start,end, period)
    return jsonDATA

In [0]:
#hide
def make_multi_LineGraph(data, logged):
  try:  
    df = data
    if logged == True:
      df['log Value'] =  np.around(np.log(df.Value), 4)
      aY = ('log Value:Q')
      scc = alt.Scale(type='log')
      color1 = alt.Color('Metric:N', legend=alt.Legend(title="Natural Log of Metrics", labelFontSize=15, titleFontSize=17),
                      scale=alt.Scale(scheme='tableau20'))
    else:
      aY = ('Value:Q')
      scc=alt.Scale(type='linear')
      color1 = alt.Color('Metric:N', legend=alt.Legend(title="Metrics",labelFontSize=15, titleFontSize=17),
                      scale=alt.Scale(scheme='tableau20'))   
    label = alt.selection_multi(fields=['Metric'], bind='legend', 
        on='mouseover',  # select on mouseover events
        nearest=True,    # select data point nearest the cursor
        empty='none'     # empty selection includes no data points
    )
    # define our base line chart of stock prices
    base = alt.Chart().mark_line().encode(
        alt.X('Year:T'),
        alt.Y(aY, scale=scc) ,
        color = color1 
        
        
        # alt.Color(' Metric:N'), opacity = alt.condition(label, alt.value(1), alt.value(0.5))

    )

    graph = alt.layer(
        base, # base line chart
        
        # add a rule mark to serve as a guide line
        alt.Chart().mark_rule(color='#aaa').encode(
            x='Year:T'
        ).transform_filter(label),
        
        # add circle marks for selected time points, hide unselected points
        base.mark_circle().encode(
            opacity=alt.condition(label, alt.value(1), alt.value(0))
        ).add_selection(label),

        # add white stroked text to provide a legible background for labels
        base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
            text=aY
        ).transform_filter(label),

        # add text labels for stock prices
        base.mark_text(align='left', dx=5, dy=-5).encode(
            text=aY
        ).transform_filter(label),
        
        data=df
    ).properties(
        width=700,
        height=400
    )
    
    return graph
  except BaseException as e:
    pass


def formatPeers(inputs):
  ColumnList = list(inputs.columns)
  yearList = list(inputs.Ticker)

  print(ColumnList)
  print(yearList)


  transposed = inputs.transpose()
  cleaned = transposed.reset_index()

  new_header = cleaned.iloc[0] #grab the first row for the header
  cleaned = cleaned[1:] #take the data less the header row
  cleaned.columns = new_header #set the header row as the df header

  byYear = cleaned.set_index("Ticker").stack()

  df = pd.DataFrame(byYear, columns=['Value'])
  df.reset_index(inplace=True)
  print(df.columns)
  df.rename(columns={0:'Ticker'}, inplace=True)
  df.dropna(inplace=True)
  df['Value'] = pd.to_numeric(df['Value'], errors='coerce')

  new_header = df.iloc[0] #grab the first row for the header
  df = df[1:] #take the data less the header row
  df.columns = new_header #set the header row as the df header


  return df

def get_financials(tickerList):
  """
  list of lists
  same order as ticker list
  each list contains a list of datframes in the order income statement , balance sheet, cash flow
  """

  tickersDF = []
  for i in tickerList:

    income_statement = get_income_statement(i)
    
    balance_sheet = get_balance_sheet(i)
    
    cash_flow = get_cash_flow(i)

    finList = [income_statement, balance_sheet, cash_flow]
    tickersDF.append(finList)

  return tickersDF

def interactive_bat(data):


  source = data
  brush = alt.selection(type='interval', encodings=['x'])

  bars = alt.Chart().mark_bar().encode(
      x='Ticker:N',
      y='mean(Value):Q',
      opacity=alt.condition(brush, alt.OpacityValue(1), alt.OpacityValue(0.7)),
  ).add_selection(
      brush
  )

  line = alt.Chart().mark_rule(color='firebrick').encode(
      y='mean(Value):Q',
      size=alt.SizeValue(3)
  ).transform_filter(
      brush
  )

  graph = alt.layer(bars, line, data=source)

  return graph

  

In [0]:
#hide
subjectCompany = "ALK"
peers = ["AAL", "DAL", 'LUV', 'UAL']
tickerList = []
tickerList.append(subjectCompany)
for i in peers:

  tickerList.append(i)

In [0]:
#hide
all_financials_querry = get_financials(tickerList)
#Works
subjectTfinancials = all_financials_querry[0]

In [601]:
#hide
## Clean some shit up for charting
from datetime import date
today = date.today()

now = str(today.strftime("%m/%d/%Y"))
print(now)

for i in subjectTfinancials:
  a = i
  a.rename({'ttm': now}, inplace=True, axis = "columns")
  a.rename({'Breakdown': "Year"}, inplace=True, axis = "columns")


04/30/2020


Note: The most recent annual column is trailing twelve months data (often denoted TTM).

In [602]:
#hide
!pip install pyfinmod
from pyfinmod.financials import Financials
parser = Financials(subjectCompany)

# (parser.cash_flow_statement)
# (parser.balance_sheet_statement)
# (parser.income_statement)


cash = (parser.cash_flow_statement).transpose().reset_index().rename(columns={'index':'Year'})
balance = (parser.balance_sheet_statement).transpose().reset_index().rename(columns={'index':'Year'})
income = (parser.income_statement).transpose().reset_index().rename(columns={'index':'Year'})# income = subjectTfinancials[0]

# income = subjectTfinancials[0]
# balance = subjectTfinancials[1]
# cash = subjectTfinancials[2]

# Subjet Company Financials

## Income Statement

In [603]:
#collapse-hide
income
## LONG FORMATED FINANCIALS (Year in columns)


row name,Year,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,Income Tax Expense,Net Income - Non-Controlling int,Net Income - Discontinued ops,Net Income,Preferred Dividends,Net Income Com,EPS,EPS Diluted,Weighted Average Shs Out,Weighted Average Shs Out (Dil),Dividend per Share,Gross Margin,EBITDA Margin,EBIT Margin,Profit Margin,Free Cash Flow margin,EBITDA,EBIT,Consolidated Income,Earnings Before Tax Margin,Net Profit Margin
0,2019-12-31,8.781000e+09,0.062561,0.000000e+00,8.781000e+09,0.0,2.370000e+09,7.718000e+09,1.063000e+09,78000000.0,1.016000e+09,247000000.0,0.0,0.0,7.690000e+08,0.0,7.690000e+08,6.240,6.19,123279000.0,124289000.0,1.391917,1.000000,0.169229,0.121057,0.087575,0.116843,1.486000e+09,1.063000e+09,7.690000e+08,0.115704,0.087575
1,2018-12-31,8.264000e+09,0.041724,0.000000e+00,8.264000e+09,0.0,2.190000e+09,7.621000e+09,6.430000e+08,91000000.0,5.850000e+08,148000000.0,0.0,0.0,4.370000e+08,0.0,4.370000e+08,3.550,3.52,123230000.0,123975000.0,1.274450,1.000000,0.125968,0.077807,0.052880,0.028437,1.041000e+09,6.430000e+08,4.370000e+08,0.070789,0.052880
2,2017-12-31,7.933000e+09,0.337548,1.950000e+08,7.738000e+09,0.0,1.924000e+09,6.673000e+09,1.260000e+09,103000000.0,1.207000e+09,173000000.0,0.0,0.0,1.034000e+09,0.0,1.034000e+09,8.390,8.35,123211000.0,123854000.0,1.194955,0.975419,0.205723,0.158830,0.130342,0.071095,1.632000e+09,1.260000e+09,1.034000e+09,0.152149,0.130342
3,2016-12-31,5.931000e+09,0.059486,1.260000e+08,5.805000e+09,0.0,1.382000e+09,4.582000e+09,1.349000e+09,55000000.0,1.345000e+09,531000000.0,0.0,0.0,8.140000e+08,0.0,8.140000e+08,6.590,6.54,123557000.0,124389000.0,1.093344,0.978756,0.288653,0.227449,0.137245,0.119373,1.712000e+09,1.349000e+09,8.140000e+08,0.226775,0.137245
4,2015-12-31,5.598000e+09,0.042846,1.130000e+08,5.485000e+09,0.0,1.254000e+09,4.300000e+09,1.298000e+09,42000000.0,1.312000e+09,464000000.0,0.0,0.0,8.480000e+08,0.0,8.480000e+08,6.610,6.56,128373000.0,129372000.0,0.788424,0.979814,0.289032,0.231869,0.151483,0.134512,1.618000e+09,1.298000e+09,8.480000e+08,0.234369,0.151483
5,2014-12-31,5.368000e+09,0.041117,9.300000e+07,5.275000e+09,0.0,1.136000e+09,4.406000e+09,9.620000e+08,48000000.0,9.750000e+08,370000000.0,0.0,0.0,6.050000e+08,0.0,6.050000e+08,4.470,4.42,135445000.0,136801000.0,0.497072,0.982675,0.233979,0.179210,0.112705,0.062593,1.256000e+09,9.620000e+08,6.050000e+08,0.181632,0.112705
6,2013-12-31,5.156000e+09,0.107151,8.400000e+07,5.072000e+09,0.0,1.086000e+09,4.318000e+09,8.380000e+08,56000000.0,8.160000e+08,308000000.0,0.0,0.0,5.080000e+08,0.0,5.080000e+08,7.260,7.16,69955000.0,70939000.0,0.394705,0.983708,0.214895,0.162529,0.098526,0.080489,1.108000e+09,8.380000e+08,5.080000e+08,0.000000,0.098526
7,2012-12-31,4.657000e+09,0.078559,7.900000e+07,4.578000e+09,0.0,1.038000e+09,4.125000e+09,5.320000e+08,64000000.0,5.140000e+08,198000000.0,0.0,0.0,3.160000e+08,0.0,3.160000e+08,4.470,4.40,70708000.0,71784000.0,0.000000,0.983036,0.170925,0.114237,0.067855,0.050462,7.960000e+08,5.320000e+08,3.160000e+08,0.110371,0.067855
8,2011-12-31,4.317800e+09,0.126686,6.720000e+07,4.250600e+09,0.0,9.905000e+08,3.868900e+09,4.489000e+08,87300000.0,3.937000e+08,149200000.0,0.0,0.0,2.445000e+08,0.0,2.445000e+08,6.810,6.66,35878000.0,36710000.0,0.000000,0.984437,0.161147,0.103965,0.056626,0.071472,6.958000e+08,4.489000e+08,2.445000e+08,0.091181,0.056626
9,2010-12-31,3.832300e+09,0.127213,5.750000e+07,3.774800e+09,0.0,0.000000e+00,3.360700e+09,4.716000e+08,108300000.0,4.059000e+08,154800000.0,0.0,0.0,2.511000e+08,0.0,2.511000e+08,7.010,6.83,35822000.0,36786000.0,0.000000,0.984996,0.183206,0.123059,0.065522,0.096730,7.021000e+08,4.716000e+08,2.511000e+08,0.105916,0.065522


In [0]:
#hide
## WIDE FORMATTED FINANCIALS (Metric In Columns)

# def formatStmnt(inputs):
#   ColumnList = list(inputs.columns)
#   yearList = list(inputs.Year)

#   print(ColumnList)
#   print(yearList)


#   transposed = inputs.transpose()
#   cleaned = transposed.reset_index()

#   new_header = cleaned.iloc[0] #grab the first row for the header
#   cleaned = cleaned[1:] #take the data less the header row
#   cleaned.columns = new_header #set the header row as the df header

#   byYear = cleaned.set_index("Year").stack()

#   df = pd.DataFrame(byYear, columns=['Value'])
#   df.reset_index(inplace=True)
#   print(df.columns)
#   df.rename(columns={0:'Metric'}, inplace=True)
#   df.dropna(inplace=True)
#   df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
#   df = inputs



#   return df


def formatStmnt(inputs):
  ColumnList = list(inputs.columns)
  yearList = list(inputs.Year)

  # print(ColumnList)
  # print(yearList)


  # transposed = inputs.transpose()
  cleaned = inputs.rename_axis('Metric')
  # print(cleaned.columns)
  # cleaned.drop(cleaned.columns[0],inplace=True)
  byYear = cleaned.set_index('Year').stack()

  df = pd.DataFrame(byYear, columns=['Value'])
  df.reset_index(inplace=True)
  # print(df.columns)
  df.rename(columns={'row name':'Metric'}, inplace=True)
  # df.dropna(inplace=True)
  # df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
  # print(df)


  # print(df.columns)
  df = df[~df['Metric'].isin(['level_0','index'])]
  # df = df.drop(['level_0','index'],axis=0)
  # print(df.Year)

  df.Year = df.Year.astype(str)
  return df

In [0]:
#hide
#stack the transposed dataframes.
df = formatStmnt(income)
metricList = df[df.columns[1]].unique()

In [0]:
#hide
####PICK METRICS


columnList = [


"Revenue"  , 
"Cost of Revenue"  , 
"Gross Profit"  , 



"R&D Expenses"  , 
"SG&A Expense"  , 
"Operating Expenses"  , 
"Operating Income"  , 
"Interest Expense"  , 



"Earnings before Tax"  , 
"Income Tax Expense"  , 

"Net Income - Non-Controlling int"  , 
"Net Income - Discontinued ops"  , 
"Net Income"  , 

"Preferred Dividends"  , 
"Net Income Com"  , 

"EPS"  , 
"EPS Diluted"  , 
"Dividend per Share"  , 

"Weighted Average Shs Out"  , 


"Weighted Average Shs Out (Dil)"  , 





"Gross Margin"  , 
"EBITDA Margin"  , 
"EBIT Margin"  , 
"Profit Margin"  , 
"Free Cash Flow margin"  , 
"EBITDA"  , 
"EBIT"  , 


"Consolidated Income"  , 
"Earnings Before Tax Margin"  , 
"Net Profit Margin"  , 

"Revenue Growth"  , 

]

revenueBases = ['Revenue','Gross Profit', "Operating Income" ,"Earnings before Tax" , "Net Income", "Net Income Com","Consolidated Income" ,"EBITDA" ,"EBIT" ]
revg = ["Revenue Growth"]
epsS = ["EPS"  , 
"EPS Diluted"  , 
"Dividend per Share"  ,]
costs = [  "Cost of Revenue"  ,"Operating Expenses" ,'Interest Expense', 'Income Tax Expense', "Preferred Dividends"  , ]

oppCosts = ["R&D Expenses"  , 
"SG&A Expense"  , ]

floats = ["Weighted Average Shs Out"  , 


"Weighted Average Shs Out (Dil)"  , 
]
margins = [
"Gross Margin"  , 
"EBITDA Margin"  , 
"EBIT Margin"  , 
"Profit Margin"  , 
"Free Cash Flow margin"  , 
"Earnings Before Tax Margin"  , 
"Net Profit Margin"  , 
]

growthRates = ["Revenue Growth"]

revs_g = df[df['Metric'].isin(revenueBases)]
eps_g = df[df['Metric'].isin(epsS)]
cost_g = df[df['Metric'].isin(costs)]
operating_g = df[df['Metric'].isin(oppCosts)]
float_g = df[df['Metric'].isin(floats)]
margins_g = df[df['Metric'].isin(margins)]

revs_gg = df[df['Metric'].isin(growthRates)]



In [0]:
#hide
most_recent_float = list((float_g.Value).dropna())[0]

### Revenues

In [608]:
#collapse-hide

make_multi_LineGraph(revs_g, False)


alt.LayerChart(...)

### Revenue Growth & Change in Growth

In [609]:
make_multi_LineGraph(revs_gg, False)


alt.LayerChart(...)

In [610]:
make_multi_LineGraph(revs_gg, True)


alt.LayerChart(...)

### EPS

In [611]:
#collapse-hide

make_multi_LineGraph(eps_g, False)



alt.LayerChart(...)

### Cost Components

In [676]:
#collapse-hide

make_multi_LineGraph(cost_g, False)


alt.LayerChart(...)

In [677]:
#collapse-hide

make_multi_LineGraph(cost_g, True)


alt.LayerChart(...)

In [646]:
#collapse-hide

make_multi_LineGraph(operating_g, False)


alt.LayerChart(...)

In [678]:
make_multi_LineGraph(operating_g, True)


alt.LayerChart(...)

### Shares Outstanding

In [614]:
#collapse-hide

make_multi_LineGraph(float_g, False)

alt.LayerChart(...)

## Margins

In [615]:
make_multi_LineGraph(margins_g, False)

alt.LayerChart(...)

## Balance Sheet

In [616]:
#collapse-hide

balance

row name,Year,Cash and cash equivalents,Short-term investments,Cash and short-term investments,Receivables,Inventories,Total current assets,"Property, Plant & Equipment Net",Goodwill and Intangible Assets,Long-term investments,Tax assets,Total non-current assets,Total assets,Payables,Short-term debt,Total current liabilities,Long-term debt,Total debt,Deferred revenue,Tax Liabilities,Deposit Liabilities,Total non-current liabilities,Total liabilities,Other comprehensive income,Retained earnings (deficit),Total shareholders equity,Investments,Net Debt,Other Assets,Other Liabilities
0,2019-12-31,221000000.0,1.300000e+09,1.521000e+09,0.0,72000000.0,2.037000e+09,6.902000e+09,2.065000e+09,3000000.0,0.0,1.095600e+10,1.299300e+10,146000000.0,269000000.0,3.201000e+09,1.264000e+09,1.533000e+09,750000000.0,40000000.0,0.0,3.451000e+09,8.662000e+09,0.0,5.133000e+09,4.331000e+09,1.303000e+09,1.312000e+09,0.0,2.010000e+09
1,2018-12-31,105000000.0,1.131000e+09,1.236000e+09,0.0,60000000.0,1.787000e+09,6.781000e+09,2.070000e+09,2000000.0,0.0,9.125000e+09,1.091200e+10,132000000.0,486000000.0,2.942000e+09,1.617000e+09,2.103000e+09,705000000.0,40000000.0,0.0,3.716000e+09,7.161000e+09,0.0,4.534000e+09,3.751000e+09,1.133000e+09,1.998000e+09,0.0,5.030000e+08
2,2017-12-31,194000000.0,1.427000e+09,1.621000e+09,0.0,57000000.0,2.146000e+09,6.284000e+09,2.076000e+09,3000000.0,0.0,8.594000e+09,1.074000e+10,120000000.0,307000000.0,2.700000e+09,2.262000e+09,2.569000e+09,699000000.0,43000000.0,0.0,3.167000e+09,7.019000e+09,0.0,4.454000e+09,3.721000e+09,1.430000e+09,2.375000e+09,0.0,1.152000e+09
3,2016-12-31,328000000.0,1.252000e+09,1.580000e+09,0.0,47000000.0,2.050000e+09,5.666000e+09,2.077000e+09,3000000.0,0.0,7.912000e+09,9.962000e+09,92000000.0,319000000.0,2.535000e+09,2.645000e+09,2.964000e+09,640000000.0,33000000.0,0.0,3.525000e+09,7.031000e+09,0.0,3.568000e+09,2.931000e+09,1.255000e+09,2.636000e+09,0.0,9.710000e+08
4,2015-12-31,73000000.0,1.255000e+09,1.328000e+09,0.0,51000000.0,1.663000e+09,4.802000e+09,0.000000e+00,2000000.0,0.0,4.870000e+09,6.533000e+09,63000000.0,115000000.0,1.806000e+09,5.710000e+08,6.860000e+08,431000000.0,22000000.0,0.0,1.615000e+09,4.122000e+09,0.0,2.890000e+09,2.411000e+09,1.257000e+09,6.130000e+08,0.0,7.010000e+08
5,2014-12-31,107000000.0,1.110000e+09,1.217000e+09,0.0,58000000.0,1.756000e+09,4.299000e+09,0.000000e+00,4000000.0,117000000.0,4.425000e+09,6.181000e+09,62000000.0,117000000.0,1.671000e+09,6.860000e+08,8.030000e+08,374000000.0,0.0,0.0,1.697000e+09,4.054000e+09,292000000.0,2.144000e+09,2.127000e+09,1.114000e+09,6.960000e+08,0.0,6.860000e+08
6,2013-12-31,80000000.0,1.250000e+09,1.330000e+09,0.0,60000000.0,1.762000e+09,3.893000e+09,0.000000e+00,4000000.0,113000000.0,4.076000e+09,5.838000e+09,64000000.0,12000000.0,1.580000e+09,7.540000e+08,7.660000e+08,335000000.0,0.0,0.0,1.475000e+09,3.809000e+09,536000000.0,1.607000e+09,2.029000e+09,1.254000e+09,6.860000e+08,0.0,7.540000e+08
7,2012-12-31,122000000.0,1.130000e+09,1.252000e+09,0.0,58000000.0,1.737000e+09,3.609000e+09,0.000000e+00,39000000.0,148000000.0,3.768000e+09,5.505000e+09,65000000.0,18000000.0,1.501000e+09,8.710000e+08,8.890000e+08,443000000.0,0.0,0.0,1.712000e+09,4.084000e+09,660000000.0,1.127000e+09,1.421000e+09,1.169000e+09,7.670000e+08,0.0,8.710000e+08
8,2011-12-31,102200000.0,1.038700e+09,1.140900e+09,0.0,44300000.0,1.595500e+09,3.401500e+09,0.000000e+00,70200000.0,134200000.0,3.599500e+09,5.195000e+09,103600000.0,31600000.0,1.509600e+09,1.099000e+09,1.130600e+09,410200000.0,71500000.0,0.0,1.413200e+09,4.021800e+09,714700000.0,8.106000e+08,1.173200e+09,1.108900e+09,1.028400e+09,0.0,1.099000e+09
9,2010-12-31,89500000.0,1.118700e+09,1.208200e+09,0.0,45100000.0,1.662000e+09,3.117100e+09,0.000000e+00,0.0,120500000.0,3.354600e+09,5.016600e+09,60200000.0,221200000.0,1.424700e+09,1.313000e+09,1.534200e+09,0.0,0.0,0.0,1.173500e+09,3.911200e+09,769500000.0,5.661000e+08,1.105400e+09,1.118700e+09,1.444700e+09,0.0,1.313000e+09


In [0]:
#hide
df = formatStmnt(balance)
metricList = df[df.columns[1]].unique()
for i in metricList:
  print(str("'"+i+"'"), ' ,')

In [0]:
#hide
columnList = [
'Cash and cash equivalents'  ,
'Short-term investments'  ,
'Cash and short-term investments'  ,
'Receivables'  ,
'Inventories'  ,
'Total current assets'  ,
'Property, Plant & Equipment Net'  ,
'Goodwill and Intangible Assets'  ,
'Long-term investments'  ,
'Tax assets'  ,
'Total non-current assets'  ,
'Total assets'  ,
'Payables'  ,
'Short-term debt'  ,
'Total current liabilities'  ,
'Long-term debt'  ,
'Total debt'  ,
'Deferred revenue'  ,
'Tax Liabilities'  ,
'Deposit Liabilities'  ,
'Total non-current liabilities'  ,
'Total liabilities'  ,
'Other comprehensive income'  ,
'Retained earnings (deficit)'  ,
'Total shareholders equity'  ,
'Investments'  ,
'Net Debt'  ,
'Other Assets'  ,
'Other Liabilities'  ,
]


assets = [
      
      'Cash and cash equivalents'  ,
'Short-term investments'  ,
'Cash and short-term investments'  ,
'Receivables'  ,
'Inventories'  ,
'Total current assets'  ,
'Property, Plant & Equipment Net'  ,
'Goodwill and Intangible Assets'  ,
'Tax assets'  ,
'Total non-current assets'  ,
'Total assets'  ,
'Other Assets'  ,
'Other Liabilities'  ,

      ]

depre = [   
'Long-term investments'  ,
'Investments'  ,
'Cash and short-term investments'  ,
]

current_liabs = [
'Payables'  ,
'Short-term debt'  ,
'Total current liabilities'  ,
'Long-term debt'  ,
'Total debt'  ,
'Deferred revenue'  ,
'Tax Liabilities'  ,
'Deposit Liabilities'  ,
'Total non-current liabilities'  ,
'Total liabilities'  ,


]


liab_equi = [              'Total liabilities',
'Total shareholders equity' ,]


equity = [ 'Common Stock',
              'Retained Earnings']

     

assets_g = df[df['Metric'].isin(assets)]
dep_g = df[df['Metric'].isin(depre)]
clab_g = df[df['Metric'].isin(current_liabs)]
llab_g = df[df['Metric'].isin(long_liabs)]
ls_g = df[df['Metric'].isin(long_short)]
e =  df[df['Metric'].isin(equity)]
le = df[df['Metric'].isin(liab_equi)]

# liabs_ass = df[df['Metric'].isin(la)]


### Assets

In [664]:
#collapse-hide

make_multi_LineGraph(assets_g, False)


alt.LayerChart(...)

In [665]:
make_multi_LineGraph(assets_g, True)


alt.LayerChart(...)

### Investment and Cash Equivalents

In [666]:
#collapse-hide

make_multi_LineGraph(dep_g, False)


alt.LayerChart(...)

In [686]:
make_multi_LineGraph(dep_g, True)


alt.LayerChart(...)

### Liabilities

In [668]:
#collapse-hide

make_multi_LineGraph(clab_g, False)

alt.LayerChart(...)

### Liabilities Vs Equity

In [672]:
#collapse-hide

make_multi_LineGraph(le, False)


alt.LayerChart(...)

In [673]:
#collapse-hide

make_multi_LineGraph(le, True)


alt.LayerChart(...)

## Cash Flows

In [627]:
#collapse-hide

cash

row name,Year,Depreciation & Amortization,Stock-based compensation,Operating Cash Flow,Capital Expenditure,Acquisitions and disposals,Investment purchases and sales,Investing Cash flow,Issuance (repayment) of debt,Issuance (buybacks) of shares,Dividend payments,Financing Cash Flow,Effect of forex changes on cash,Net cash flow / Change in cash,Free Cash Flow,Net Cash/Marketcap
0,2019-12-31,423000000.0,29000000.0,1.722000e+09,6.960000e+08,-0.000000e+00,-185000000.0,-7.910000e+08,-1.100000e+09,-75000000.0,-173000000.0,-8.130000e+08,0.0,221000000.0,1.026000e+09,0.0000
1,2018-12-31,398000000.0,47000000.0,1.195000e+09,9.600000e+08,-0.000000e+00,232000000.0,-6.310000e+08,-8.070000e+08,-50000000.0,-158000000.0,-6.470000e+08,0.0,105000000.0,2.350000e+08,0.0000
2,2017-12-31,372000000.0,55000000.0,1.590000e+09,1.026000e+09,-0.000000e+00,-256000000.0,-1.132000e+09,-3.970000e+08,-75000000.0,-148000000.0,-5.920000e+08,0.0,-134000000.0,5.640000e+08,0.0000
3,2016-12-31,363000000.0,26000000.0,1.386000e+09,6.780000e+08,-1.951000e+09,-191000000.0,-2.622000e+09,-2.490000e+08,-193000000.0,-136000000.0,1.491000e+09,0.0,255000000.0,7.080000e+08,0.0000
4,2015-12-31,320000000.0,25000000.0,1.584000e+09,8.310000e+08,-0.000000e+00,-657000000.0,-9.300000e+08,-1.160000e+08,-505000000.0,-102000000.0,-6.880000e+08,0.0,-34000000.0,7.530000e+08,0.0000
5,2014-12-31,294000000.0,6000000.0,1.030000e+09,6.940000e+08,-0.000000e+00,-205000000.0,-5.410000e+08,-1.190000e+08,-321000000.0,-68000000.0,-4.620000e+08,0.0,27000000.0,3.360000e+08,0.0000
6,2013-12-31,270000000.0,32000000.0,9.810000e+08,5.660000e+08,-0.000000e+00,-288000000.0,-6.980000e+08,-1.610000e+08,-135000000.0,-28000000.0,-3.250000e+08,0.0,-42000000.0,4.150000e+08,0.0000
7,2012-12-31,264000000.0,10000000.0,7.530000e+08,5.180000e+08,-0.000000e+00,-142000000.0,-6.450000e+08,-2.750000e+08,-29000000.0,-0.0,-8.800000e+07,0.0,20000000.0,2.350000e+08,0.0000
8,2011-12-31,246900000.0,16500000.0,6.960000e+08,3.874000e+08,-0.000000e+00,-7300000.0,-4.033000e+08,-3.335000e+08,-60100000.0,-0.0,-2.800000e+08,0.0,12700000.0,3.086000e+08,0.0000
9,2010-12-31,230500000.0,0.0,5.537000e+08,1.830000e+08,-0.000000e+00,-136100000.0,-2.952000e+08,-3.210000e+08,-8600000.0,-0.0,-3.332000e+08,0.0,-74700000.0,3.707000e+08,0.0000


In [690]:
#hide
df = formatStmnt(cash)
metricList = df[df.columns[1]].unique()

for i in metricList:
  print(str('"'+i+'"'),',')


"Depreciation & Amortization" ,
"Stock-based compensation" ,
"Operating Cash Flow" ,
"Capital Expenditure" ,
"Acquisitions and disposals" ,
"Investment purchases and sales" ,
"Investing Cash flow" ,
"Issuance (repayment) of debt" ,
"Issuance (buybacks) of shares" ,
"Dividend payments" ,
"Financing Cash Flow" ,
"Effect of forex changes on cash" ,
"Net cash flow / Change in cash" ,
"Free Cash Flow" ,
"Net Cash/Marketcap" ,


In [0]:
vects = [
         "Depreciation & Amortization" ,
"Stock-based compensation" ,
"Operating Cash Flow" ,
"Capital Expenditure" ,
"Acquisitions and disposals" ,
"Investment purchases and sales" ,
"Investing Cash flow" ,
"Issuance (repayment) of debt" ,
"Issuance (buybacks) of shares" ,
"Dividend payments" ,


"Financing Cash Flow" ,


"Effect of forex changes on cash" ,
"Net cash flow / Change in cash" ,
"Free Cash Flow" ,
"Net Cash/Marketcap" ,


]


outflows = [ "Depreciation & Amortization" ,
"Stock-based compensation" ,"Dividend payments" ,"Capital Expenditure" ,"Issuance (repayment) of debt" ,
"Issuance (buybacks) of shares" ,"Effect of forex changes on cash" ,]

flows = ["Operating Cash Flow" ,"Acquisitions and disposals" ,
"Investment purchases and sales" , "Investing Cash flow" ,"Financing Cash Flow" ,"Free Cash Flow" ,
]

ratios = ["Net cash flow / Change in cash" ,"Net Cash/Marketcap" ,


]

     

out_g = df[df['Metric'].isin(outflows)]
flow_g = df[df['Metric'].isin(flows)]
ratio_g = df[df['Metric'].isin(ratios)]

### Outflows





In [692]:
make_multi_LineGraph(out_g, False)

alt.LayerChart(...)

In [693]:
make_multi_LineGraph(out_g, True)

alt.LayerChart(...)

### Other Cash Flows


In [694]:
make_multi_LineGraph(flow_g, False)

alt.LayerChart(...)

In [696]:
make_multi_LineGraph(flow_g, True)

alt.LayerChart(...)

### Ratios

In [697]:
make_multi_LineGraph(ratio_g, False)

alt.LayerChart(...)

In [699]:
make_multi_LineGraph(ratio_g, True)

alt.LayerChart(...)

# Peer Comparison

In [628]:
#hide
import sys
import requests
import bs4
res = requests.get('https://en.wikipedia.org/wiki/Russell_1000_Index')
# res = requests.get("http://www.nationmaster.com/country-info/stats/Media/Internet-users")
soup = BeautifulSoup(res.content,'lxml')
a1 = soup.find_all('table')
getReturns = a1[2]
getTickers = a1[3]
# print(a1)



df1 = pd.read_html(str(getReturns))
AnnualReturns = df1[0]
# a1 = soup.find_all('table')[3]
df2 = pd.read_html(str(getTickers))
print(len(df2))
russ1kList = df2[0]



1


In [0]:
#hide
IWB_comps = list(russ1kList['Ticker'])

In [0]:
#hide
import random
randomSample = []

# simuls = 30


# for i in range(simuls):
#   secure_random = random.SystemRandom()
#   a = secure_random.choice(IWB_comps)
#   randomSample.append(a)
#   # print(a)
# for i in tickerList:
#   randomSample.append(i)

In [0]:
#hide
yfStatsComp = yf_summaryTable(randomSample)
sumTable = pd.DataFrame.from_records(yfStatsComp)

In [632]:
#hide
new = sumTable.transpose()
new.columns

Index([], dtype='object')

In [0]:
#hide
vects= ['Index', 
        'P/E', 
        'EPS (ttm)', 
        'Insider Own',
        'Shs Outstand',
        'Perf Week',
        'Market Cap', 
        'Forward P/E',
        'EPS next Y',
        'Insider Trans',
        'Shs Float', 
        'Perf Month',
        'Income',
        'PEG',
        'EPS next Q',
        'Inst Own', 
        'Short Float',
        'Perf Quarter',
        'Sales',
        'P/S',
        'EPS this Y',
        'Inst Trans',
        'Short Ratio',
        'Perf Half Y',
        'Book/sh',
        'P/B',
        'ROA',
        'Target Price',
        'Perf Year',
        'Cash/sh',
        'P/C',
        'EPS next 5Y',
        'ROE',
        '52W Range',
        'Perf YTD', 
        'Dividend', 
        'P/FCF',
        'EPS past 5Y', 
        'ROI',
        '52W High',
        'Beta', 
        'Dividend %',
        'Quick Ratio',
        'Sales past 5Y',
        'Gross Margin',
        '52W Low',
        'ATR',
        'Employees',
        'Current Ratio',
        'Sales Q/Q',
        'Oper. Margin',
        'RSI (14)',
        'Volatility',
        'Optionable',
        'Debt/Eq',
        'EPS Q/Q', 
        'Profit Margin',
        'Rel Volume',
        'Prev Close',
        'Shortable',
        'LT Debt/Eq',
        'Earnings',
        'Payout',
        'Avg Volume',
        'Price',
        'Recom', 
        'SMA20',
        'SMA50',
        'SMA200',
        'Volume',
        'Change']

In [634]:
#hide
!pip install finviz
import finviz
recs = []
xyz= 0
for i in tickerList:
    try:
      hold = [i]
      for b in (finviz.get_stock(i)):
        colls = (list((finviz.get_stock(i)).keys()))
        hold.append(finviz.get_stock(i)[b])
      recs.append(hold)
    except BaseException as e:
      print(e, "could not find", i)
print(colls)


['Company', 'Sector', 'Industry', 'Country', 'Index', 'P/E', 'EPS (ttm)', 'Insider Own', 'Shs Outstand', 'Perf Week', 'Market Cap', 'Forward P/E', 'EPS next Y', 'Insider Trans', 'Shs Float', 'Perf Month', 'Income', 'PEG', 'EPS next Q', 'Inst Own', 'Short Float', 'Perf Quarter', 'Sales', 'P/S', 'EPS this Y', 'Inst Trans', 'Short Ratio', 'Perf Half Y', 'Book/sh', 'P/B', 'ROA', 'Target Price', 'Perf Year', 'Cash/sh', 'P/C', 'EPS next 5Y', 'ROE', '52W Range', 'Perf YTD', 'Dividend', 'P/FCF', 'EPS past 5Y', 'ROI', '52W High', 'Beta', 'Dividend %', 'Quick Ratio', 'Sales past 5Y', 'Gross Margin', '52W Low', 'ATR', 'Employees', 'Current Ratio', 'Sales Q/Q', 'Oper. Margin', 'RSI (14)', 'Volatility', 'Optionable', 'Debt/Eq', 'EPS Q/Q', 'Profit Margin', 'Rel Volume', 'Prev Close', 'Shortable', 'LT Debt/Eq', 'Earnings', 'Payout', 'Avg Volume', 'Price', 'Recom', 'SMA20', 'SMA50', 'SMA200', 'Volume', 'Change']


In [635]:
#hide
listCols = (list(['Ticker']))
for i in colls:
  # print(i)
  listCols.append(i)

print(listCols)
bigTable = pd.DataFrame.from_records(recs, columns=listCols)


['Ticker', 'Company', 'Sector', 'Industry', 'Country', 'Index', 'P/E', 'EPS (ttm)', 'Insider Own', 'Shs Outstand', 'Perf Week', 'Market Cap', 'Forward P/E', 'EPS next Y', 'Insider Trans', 'Shs Float', 'Perf Month', 'Income', 'PEG', 'EPS next Q', 'Inst Own', 'Short Float', 'Perf Quarter', 'Sales', 'P/S', 'EPS this Y', 'Inst Trans', 'Short Ratio', 'Perf Half Y', 'Book/sh', 'P/B', 'ROA', 'Target Price', 'Perf Year', 'Cash/sh', 'P/C', 'EPS next 5Y', 'ROE', '52W Range', 'Perf YTD', 'Dividend', 'P/FCF', 'EPS past 5Y', 'ROI', '52W High', 'Beta', 'Dividend %', 'Quick Ratio', 'Sales past 5Y', 'Gross Margin', '52W Low', 'ATR', 'Employees', 'Current Ratio', 'Sales Q/Q', 'Oper. Margin', 'RSI (14)', 'Volatility', 'Optionable', 'Debt/Eq', 'EPS Q/Q', 'Profit Margin', 'Rel Volume', 'Prev Close', 'Shortable', 'LT Debt/Eq', 'Earnings', 'Payout', 'Avg Volume', 'Price', 'Recom', 'SMA20', 'SMA50', 'SMA200', 'Volume', 'Change']


## Big Comp Table

In [636]:
#collapse-hide

bigTable

,Ticker,Company,Sector,Industry,Country,Index,P/E,EPS (ttm),Insider Own,Shs Outstand,Perf Week,Market Cap,Forward P/E,EPS next Y,Insider Trans,Shs Float,Perf Month,Income,PEG,EPS next Q,Inst Own,Short Float,Perf Quarter,Sales,P/S,EPS this Y,Inst Trans,Short Ratio,Perf Half Y,Book/sh,P/B,ROA,Target Price,Perf Year,Cash/sh,P/C,EPS next 5Y,ROE,52W Range,Perf YTD,Dividend,P/FCF,EPS past 5Y,ROI,52W High,Beta,Dividend %,Quick Ratio,Sales past 5Y,Gross Margin,52W Low,ATR,Employees,Current Ratio,Sales Q/Q,Oper. Margin,RSI (14),Volatility,Optionable,Debt/Eq,EPS Q/Q,Profit Margin,Rel Volume,Prev Close,Shortable,LT Debt/Eq,Earnings,Payout,Avg Volume,Price,Recom,SMA20,SMA50,SMA200,Volume,Change
0,ALK,"Alaska Air Group, Inc.",Services,Regional Airlines,USA,S&P 500,5.50,6.18,0.30%,123.13M,23.50%,4.19B,8.20,216.00%,-1.80%,122.05M,20.01%,769.00M,4.44,-1.08,90.00%,3.27%,-49.17%,8.78B,0.48,75.50%,-0.59%,1.40,-51.48%,35.18,0.97,6.00%,45.25,-45.07%,12.35,2.75,1.24%,18.90%,20.02 - 72.22,-49.82%,-,3.02,6.90%,14.00%,-52.92%,1.42,-,0.60,10.30%,56.10%,69.83%,2.92,24134,0.60,7.90%,12.10%,54.94,6.89% 8.15%,Yes,0.35,683.70%,8.80%,1.40,31.58,Yes,0.29,May 05 BMO,22.40%,2.85M,34.00,2.30,19.27%,-9.09%,-42.09%,"3,985,866",7.66%
1,AAL,American Airlines Group Inc.,Services,Major Airlines,USA,S&P 500,3.32,3.81,1.30%,426.89M,22.98%,5.39B,11.03,108.50%,0.98%,421.02M,3.10%,1.69B,0.29,-2.26,84.30%,19.37%,-52.87%,45.77B,0.12,25.20%,-0.88%,1.64,-59.73%,-0.27,-,2.80%,16.25,-63.05%,8.96,1.41,11.28%,-,9.09 - 34.99,-55.96%,0.40,-,-0.70%,10.30%,-63.90%,1.81,3.17%,0.30,1.40%,61.50%,38.94%,1.36,133000,0.40,3.40%,6.70%,51.95,7.34% 9.51%,Yes,-,35.10%,3.70%,2.21,11.26,Yes,-,Apr 30 BMO,10.50%,49.85M,12.63,3.20,15.61%,-15.87%,-49.70%,"110,358,587",12.17%
2,DAL,"Delta Air Lines, Inc.",Services,Major Airlines,USA,S&P 500,5.07,5.39,0.30%,639.81M,21.58%,17.48B,6.61,170.10%,-86.23%,639.81M,-4.71%,3.50B,1.32,-4.17,88.70%,3.91%,-51.77%,45.13B,0.39,28.70%,-0.15%,0.71,-50.48%,22.46,1.22,5.40%,38.93,-53.13%,-,-,3.85%,23.90%,19.10 - 63.44,-53.28%,-,4.14,56.40%,19.60%,-56.93%,1.33,-,0.50,3.10%,53.50%,43.04%,2.54,91224,0.50,-18.00%,11.50%,50.93,6.29% 8.27%,Yes,1.19,-176.60%,7.80%,2.27,24.34,Yes,0.88,Apr 22 BMO,28.80%,35.19M,27.32,2.20,16.37%,-19.29%,-47.26%,"80,009,047",12.24%
3,LUV,Southwest Airlines Co.,Services,Regional Airlines,USA,S&P 500,7.34,4.22,0.30%,526.00M,4.24%,16.29B,9.87,190.80%,-59.39%,506.62M,-11.77%,2.27B,4.59,-0.41,89.10%,1.91%,-45.56%,22.43B,0.73,-2.00%,0.57%,0.95,-46.27%,18.69,1.66,8.70%,43.06,-42.89%,7.74,4.00,1.60%,23.30%,28.25 - 58.83,-42.63%,0.72,6.29,20.80%,18.20%,-47.36%,1.40,2.32%,0.60,3.80%,74.50%,9.63%,2.56,60800,0.70,0.50%,13.20%,42.21,5.85% 6.69%,Yes,0.27,-21.80%,10.30%,9.86,29.69,Yes,0.19,Apr 28 BMO,16.40%,10.18M,30.97,2.60,-2.73%,-20.46%,-38.70%,"100,371,268",4.31%
4,UAL,"United Airlines Holdings, Inc.",Services,Major Airlines,USA,S&P 500,2.68,11.63,0.30%,248.79M,20.60%,7.76B,5.74,134.70%,-0.27%,246.21M,3.48%,3.01B,0.26,-2.88,-,9.11%,-58.50%,43.26B,0.18,51.30%,0.22%,1.06,-66.15%,45.72,0.68,5.80%,49.50,-64.88%,19.87,1.57,10.20%,28.00%,17.80 - 96.03,-64.57%,-,3.26,31.60%,12.90%,-67.50%,1.56,-,0.50,2.10%,62.00%,75.34%,3.49,96000,0.50,3.80%,9.90%,48.35,6.98% 10.68%,Yes,1.29,51.20%,7.00%,2.44,27.84,Yes,1.16,Apr 30 AMC,0.00%,21.26M,31.21,2.40,14.40%,-23.51%,-58.95%,"51,784,724",12.10%


In [0]:
#hide
# import numpy as np 
# from pandas import DataFrame
# import matplotlib.pyplot as plt


# plt.pcolor(bigTable)
# plt.yticks(np.arange(0.5, len(df.index), 1), bigTable.index)
# plt.xticks(np.arange(0.5, len(df.columns), 1), bigTable.columns)
# plt.show()

In [0]:
  # hide
  # new_header = df.iloc[0] #grab the first row for the header
  # df = df[1:] #take the data less the header row
  # df.columns = new_header #set the header row as the df header 

In [0]:
#hide
#PE data http://www.stern.nyu.edu/~adamodar/pc/datasets/pedata.xls
#
#
#
#

Market Value

Enterprise Value

Leverage

# DCF Valuation

In [640]:
#hide
!pip install pyfinmod

In [0]:
#hide


from pyfinmod.financials import Financials
from pyfinmod.ev import fcf, enterprise_value

from pyfinmod.financials import Financials

from pyfinmod.financials import Financials
from pyfinmod.wacc import total_debt

from pyfinmod.financials import Financials
from pyfinmod.wacc import tax_rate





from pyfinmod.financials import Financials
from pyfinmod.wacc import cost_of_debt

from pyfinmod.financials import Financials
from pyfinmod.wacc import cost_of_equity

# RFR = .01
# mkt_return = .09
# LR_g =.1
# SR_g = .1 + LR_g

# print((RFR,mkt_return,SR_g,LR_g))




# lec = (new.loc['Weighted Average Shs Out'][0])

# eq = parser.balance_sheet_statement.loc['Total liabilities'][0]




# Calculate WACC


parser = Financials(subjectCompany)
from pyfinmod.wacc import wacc

from pyfinmod.ev import dcf, fcf
from pyfinmod.financials import Financials

def fcf_dcf_using_wacc(RFR,mkt_return,SR_g,LR_g, shock, subjectCompany=subjectCompany):
  # print(fcf_dcf_using_wacc(RFR,mkt_return,SR_g,LR_g))
  parser = Financials(subjectCompany)
  lec = (parser.balance_sheet_statement.iloc[-5::5])

  for i in lec.itertuples():
    ec = (i._1)
    break


  parser = Financials(subjectCompany)
  parser.income_statement.columns
  currentPrice = parser.price


  parser = Financials(subjectCompany)
  ev = enterprise_value(parser.balance_sheet_statement)[0]
  ecToev = ec / ev
  equitys_share = (1-ecToev)






  parser = Financials(subjectCompany)
  fcf_value = fcf(parser.cash_flow_statement)
  total_debt(parser.balance_sheet_statement)
  tax_rate(parser.income_statement)
  parser.mktCap



  # cost_of_debt(parser.balance_sheet_statement, parser.income_statement)

  # cost_of_equity(float(parser.beta), risk_free_interest_rate = RFR, market_return = mkt_return)
  parser = Financials(subjectCompany)

  aapl_wacc = wacc(parser.mktCap,
                  parser.balance_sheet_statement,
                  parser.income_statement,
                  float(parser.beta),
                  risk_free_interest_rate=RFR,
                  market_return=mkt_return)
  

  parser = Financials(subjectCompany)
  if shock:
    output = dcf_n1Shock(fcf(parser.cash_flow_statement), aapl_wacc, shock, short_term_growth=SR_g, long_term_growth=LR_g)
  else:
    output = dcf(fcf(parser.cash_flow_statement), aapl_wacc, short_term_growth=SR_g, long_term_growth=LR_g)
  
  value_per_share = output / (parser.income_statement.loc['Weighted Average Shs Out'][0])

  pps = equitys_share * value_per_share

  IR = ((pps - currentPrice) / currentPrice)*100

  out = (value_per_share, pps, SR_g, LR_g, IR)
  return (out)
  # return float(value_per_share)


Generating a range of growth rate scenarios:

In [0]:
#collapse-hide

SR_g_scenarios = np.linspace(-.03,.04,15,endpoint=False)





### DCF Growth Rate Scenarios

In [643]:
#hide
parser = Financials(subjectCompany)
parser.income_statement.columns

currentPrice = parser.price
colz = ('Firm Value/Share ($)','Equity Value/Share ($)', 'SR CAGR', 'LR CAGR', 'Implied ROI (%)')

records_dcf_1 = [colz]
print("Current Price:", currentPrice)
RFR = .02
mkt_return = .12
for i in range(len(SR_g_scenarios)):
  SR_g = SR_g_scenarios[i]
  LR_g = .01
  records_dcf_1.append(fcf_dcf_using_wacc(RFR,mkt_return,SR_g,LR_g, False))
  # print(records_dcf_1)
  print("Done computing ", (i + 1), ' DCF out of ',len(SR_g_scenarios))

dcf_growth_rates_outputs = pd.DataFrame.from_records(records_dcf_1, columns = list(records_dcf_1[0]))



Current Price: 34.0
Done computing  1  DCF out of  15
Done computing  2  DCF out of  15
Done computing  3  DCF out of  15
Done computing  4  DCF out of  15
Done computing  5  DCF out of  15
Done computing  6  DCF out of  15
Done computing  7  DCF out of  15
Done computing  8  DCF out of  15
Done computing  9  DCF out of  15
Done computing  10  DCF out of  15
Done computing  11  DCF out of  15
Done computing  12  DCF out of  15
Done computing  13  DCF out of  15
Done computing  14  DCF out of  15
Done computing  15  DCF out of  15


In [644]:
#collapse-hide

ind = (dcf_growth_rates_outputs.columns[4])
dcf_growth_rates_outputs


,Firm Value/Share ($),Equity Value/Share ($),SR CAGR,LR CAGR,Implied ROI (%)
0,Firm Value/Share ($),Equity Value/Share ($),SR CAGR,LR CAGR,Implied ROI (%)
1,138.905,77.4672,-0.03,0.01,127.845
2,141.907,79.1414,-0.0253333,0.01,132.769
3,144.963,80.846,-0.0206667,0.01,137.782
4,148.075,82.5815,-0.016,0.01,142.887
5,151.243,84.3483,-0.0113333,0.01,148.083
6,154.468,86.147,-0.00666667,0.01,153.373
7,157.751,87.9778,-0.002,0.01,158.758
8,161.092,89.8412,0.00266667,0.01,164.239
9,164.493,91.7378,0.00733333,0.01,169.817


Seems to do a decent job, however this does not take into account the shock from coronavirus which will probably hurt revenues next year. We will modifiy the discounted cash flow formula to include a shock to FCF in the first year.

In [648]:
#hide
from pyfinmod.basic import npv
from math import sqrt

def dcf_n1Shock(fcfs, wacc, SHOCK, short_term_growth, long_term_growth):
    latest_fcf_date = fcfs.index.max()
    dates = pd.date_range(latest_fcf_date, periods=6, freq="365D")[1:]
    future_cash_flows = [fcfs[latest_fcf_date]]
    for i in range(5): # 5?
        # print(i)
        # print(SHOCK)
        if i == 0:        
          next_year_fcf = future_cash_flows[-1] * (1 + short_term_growth)
          future_cash_flows.append(next_year_fcf*SHOCK)

        else:
            
          next_year_fcf = future_cash_flows[-1] * (1 + short_term_growth)
          future_cash_flows.append(next_year_fcf)



    future_cash_flows = future_cash_flows[1:]
    df = pd.DataFrame(data={"fcf": future_cash_flows, "date": dates})
    # df.set_index('date', inplace=True)
    df["terminal value"] = 0
    last_index = df.index[-1]
    last_short_term_fcf = df.at[last_index, "fcf"]
    df.at[last_index, "terminal value"] = (
        last_short_term_fcf * (1 + long_term_growth) / (wacc - long_term_growth)
    )
    df["cash flow"] = df[["fcf", "terminal value"]].apply(sum, axis=1)
    return npv(df, wacc) * sqrt((1 + wacc))

parser = Financials(subjectCompany)
parser.income_statement.columns

currentPrice = parser.price
colz = ('Firm Value/Share ($)','Equity Value/Share ($)', 'SR CAGR', 'LR CAGR', 'Implied ROI (%)')

records_dcf_1 = [colz]
print("Current Price:", currentPrice)
RFR = .02
mkt_return = .12
for i in range(len(SR_g_scenarios)):
  SR_g = SR_g_scenarios[i]
  LR_g = .01
  records_dcf_1.append(fcf_dcf_using_wacc(RFR,mkt_return,SR_g,LR_g, 1/2))
  # print(records_dcf_1)
  print("Done computing ", (i + 1), ' DCF out of ',len(SR_g_scenarios))

dcf_growth_rates_outputs_shock = pd.DataFrame.from_records(records_dcf_1, columns = list(records_dcf_1[0]))



Current Price: 34.0


KeyboardInterrupt: ignored

In [0]:
dcf_growth_rates_outputs_shock

## DCF Peer Comparison

In [0]:

parser = Financials(subjectCompany)
parser.income_statement.columns

currentPrice = parser.price
colz = ('Firm Value/Share ($)','Equity Value/Share ($)', 'SR CAGR', 'LR CAGR', 'Implied ROI (%)')

records_dcf_2 = []
print("Current Price:", currentPrice)
RFR = .02
mkt_return = .12
LR_g = .01
SR_g = .02
for i in tickerList:
  records_dcf_2.append(fcf_dcf_using_wacc(RFR,mkt_return,SR_g,LR_g, .5, i))
  # print(records_dcf_1)


peer_dcf = pd.DataFrame.from_records(records_dcf_2, columns = list(records_dcf_1[0]))

peer_dcf['ticker'] = tickerList
peer_dcf.set_index('ticker', inplace=True)

In [0]:
peer_dcf

## Historical DCF Scenario Backtest Analysis Using Differint Bases (Work In Progress.)

(WIP -- NEED TO DO MORE CODE BUT THIS WILL HOW HOW ACCURATE OUR MODELS ARE OVER TIME AND WHAT EARNINGS BASE MAKE THE BEST ESTIMATORS FOR THIS FIRM)

In [0]:
from google.colab import drive
import sys

# Conclusions and Summary

In [0]:
!pip install matplotlib urllib3 seaborn
!git clone https://github.com/halessi/DCF.git
!ls DCF
!python DCF/main.py --t ALK --i 'quarter' --y 3 --eg .15 --steps 2 --s 0.1 --v eg